want to try and optimize feature selection

from what i can see, accident, vehicle, and person datasets seem the most relevant, might try to take some features from there

converting each csv into a dataframe for better handling
- df is our base df, will be writing it to a new csv when done
- finding intersection btwn all three csvs, using that to drop features from df1 and df2.
- i then will be adding df1 and df2 (with only unique cols) into df
- from here, determine features with strongest correlation to target since our max is 20 features
- going to run decision tree model from demo.ipynb on new csv to see if any improvements in accuracy

In [ ]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
RANDOM_STATE = 42
files = ["accident23", "person23", "vehicle23"]

df = pd.read_csv("C:\\Users\\15013\\Desktop\\CRSS2023CSV\\accident23.csv")
df1 = pd.read_csv("C:\\Users\\15013\\Desktop\\CRSS2023CSV\\vehicle23.csv")
df2 = pd.read_csv("C:\\Users\\15013\\Desktop\\CRSS2023CSV\\person23.csv")

df3 = pd.DataFrame()

C:\Users\15013\AppData\Local\Temp\ipykernel_18008\447166427.py:9: DtypeWarning: Columns (57,59,69,73) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("C:\\Users\\15013\\Desktop\\CRSS2023CSV\\vehicle23.csv")


In [ ]:
accidents_col = list(df.columns)
vehicles_col = list(df1.columns)
person_col = list(df2.columns)

intersect = []

for x in accidents_col: #there has to be a better way to do this.. 
    if x in vehicles_col:
        if x in person_col:
            intersect.append(x) #if feature is in all 3 dfs

intersect.remove("CASENUM") #we wanna keep casenum since this is our joining feature
#gonna use this list to drop features from df1 and df2

df1 = df1.drop(columns = intersect, errors = "ignore")
df2 = df2.drop(columns = intersect, errors = "ignore")

print(len(df))
print(len(df1))
print(len(df2))

50103
87461
122388


In [28]:
#merge all three dfs based on casenum
df3 = (
    df
    .merge(df1, on="CASENUM", how="left")
    .merge(df2, on="CASENUM", how="left")
)

df3 = df3[df3['MAX_SEV'] <= 4] #drop where sev vals > 4

#print(df3.columns) #far too many columns
y = df3['MAX_SEV'] #target feature
X = df3.drop(columns = ["MAX_SEV"])


print(df3['AGE'].value_counts(ascending = True))

AGE
115.0        1
102.0        2
106.0        2
100.0        3
101.0        4
         ...  
18.0      5413
22.0      5427
20.0      5430
19.0      5604
998.0    10338
Name: count, Length: 107, dtype: int64


In [ ]:
# checking for missing/unknown values 
#most of the code taken from demo.ipynb

UNKNOWN_MAP = {
    #accident csv

}

acc_cols = [
    "MONTH",
    "DAY_WEEK",
    "HOUR",
    "WEATHER",
    "LIGHT_COND",
    "HARM_EV",
    "MAN_COLL",
    "REL_ROAD",
    "TYP_INT",
    "URBANICITY"
]

veh_cols = [
    "VSPD_LIM",
    "VSURCOND",
    "ACC_TYPE",
    "SPEEDREL",
    "DEFORMED"
]

person_cols = [
    "AGE",
    "SEX",
    "PER_TYPE",
    "SEAT_POS",
    "REST_USE"
]

